In [1]:
from PIL import Image
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.utils import save_image
import os
import matplotlib.pyplot as plt
from torchvision import models
from tqdm.notebook import tqdm
import numpy as np


In [2]:
class AdversarialDataset(Dataset):
    def __init__(self, annotation_file, categories_file, img_dir, x_transform=None, y_transform=None):
        self.img_dir = img_dir
        annotations = pd.read_csv(annotation_file)
        self.categories = pd.read_csv(categories_file)
        self.images = annotations["ImageId"]
        self.labels = annotations["TrueLabel"]
        self.x_transform = x_transform
        self.y_transform = y_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx] + ".png")
        image = Image.open(img_path)

        if self.x_transform:
            image = self.x_transform(image)

        return image, self.labels[idx] - 1, self.images[idx]

In [3]:
class TensorToDevice(object):
    def __init__(self, device):
        self.device = device
        
    def __call__(self, image):
        image = image.to(self.device)
        return image

In [4]:
def show_image(datarow):
    permuted = torch.permute(datarow[0], (1, 2, 0))
    plt.title(datarow[1])
    plt.imshow(permuted)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

transform = transforms.Compose([
    transforms.ToTensor(),
    TensorToDevice(device),
    transforms.Normalize(mean=mean, std=std)
])

In [7]:
dataset = AdversarialDataset("images.csv", "categories.csv", "images", x_transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=False) #, pin_memory=True)

model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
model = model.to(device)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

<h1>Training</h1>

In [8]:
n_epochs = 0
running_loss = 0
n_correct = 0

for epoch in range(n_epochs):
    for x_batch, y_batch in tqdm(dataloader):
        x_batch = x_batch.float()
        x_batch = x_batch.to(device)
        y_batch = y_batch.type(torch.LongTensor)
        y_batch = y_batch.to(device)
    
        output, _ = model(x_batch)
    
        loss = loss_function(output, y_batch)
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item() + x_batch.size(0)
    
        _, preds = torch.max(output, dim=1)
        n_correct += torch.sum(preds == y_batch.data)

<h1>Testing</h1>

In [9]:
# n_correct = 0
# 
# for x_batch, y_batch in tqdm(dataloader):
#     x_batch = x_batch.to(device)
#     y_batch = y_batch.to(device)
# 
#     output, _ = model(x_batch)
# 
#     _, preds = torch.max(output, dim=1)
#     n_correct += torch.sum(preds == y_batch)
#     
# print(n_correct/len(dataset))

<h3> Generate C&W images </h3>

In [8]:
import cw_impl.cw as cw

def generate_cw_samples(model, dataloader, save_to_disk=True, save_dir="adversarial_images_gpu"):
    inputs_box = (min((0 - m) / s for m, s in zip(mean, std)), max((1 - m) / s for m, s in zip(mean, std)))
    
    adversary = cw.L2Adversary(targeted=False, 
                               confidence=0.0,
                               search_steps=10,
                               abort_early=True,
                               optimizer_lr=5e-4,
                               box=inputs_box)
    
    model.eval()
    for inputs, targets, input_ids in tqdm(dataloader):
        # inputs = inputs.to(device)
        # targets = targets.to(device)
        
        adversarial_examples = adversary(model, inputs, targets, to_numpy=False)
        
        if save_to_disk:
            for i in range(adversarial_examples.shape[0]):
                # Save as numpy array instead of .png to work around loss of data when scaling
                with open(os.path.join(save_dir, input_ids[i] + ".npy"), 'wb') as f:
                    np.save(f, adversarial_examples[i])
        
generate_cw_samples(model, dataloader)
        

  0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
def test_batch(inputs, outputs, device):
    inputs = inputs.to(device)
    outputs = outputs.to(device)
    
    output = model(inputs)

    _, preds = torch.max(output, dim=1)
    print(preds)
    return torch.sum(preds == outputs)/inputs.shape[0]